In [1]:
import sqlite3
import random
import string

database_path = '/home/myrto/pink_dir/github/DataFiles/decem_17_db.db'

conn = sqlite3.connect(database_path)
cursor = conn.cursor()

cursor.execute("""SELECT subreddit FROM parents WHERE score>=130 GROUP BY subreddit HAVING SUM(length)>=1000""")
ps = cursor.fetchall()
print(len(ps))

cursor.execute("""SELECT * FROM parents WHERE score>=130""")
ps2 = cursor.fetchall()
print(len(ps2))

928
584828


In [2]:
labels_list=[]

for k in ps:
    count_words = 0
    num_doc = 0
    for parent_id, reply_id, parent_body, parent_subreddit, parent_utc_time, parent_score, parent_length in ps2:
        if parent_subreddit == (str(k)[2:-3]):
            cursor.execute("""SELECT * FROM replies WHERE id='{}'""".format(reply_id))
            reply = cursor.fetchone()
            if reply:
                reply_id, reply_body, reply_subreddit, reply_utc_time, reply_score, reply_length = reply
                to_add = parent_body + reply_body
                if not ('http' in to_add):
                    count_words = count_words + parent_length + reply_length
        if count_words >= 2000:
            count_words = 0
            num_doc += 1
    if num_doc > 50:
        new_label = str(k)[2:-3] 
        labels_list.append(new_label)
        print(new_label)
print(labels_list)
print(len(labels_list))

AdviceAnimals
AskMen
AskReddit
BlackPeopleTwitter
CFB
DestinyTheGame
Games
IAmA
JUSTNOMIL
MMA
Overwatch
Showerthoughts
SquaredCircle
StarWars
The_Donald
WTF
WritingPrompts
aww
funny
gaming
gifs
hearthstone
hiphopheads
leagueoflegends
legaladvice
mildlyinteresting
movies
nba
news
nfl
nottheonion
pics
politics
relationships
soccer
technology
television
todayilearned
videos
worldnews
['AdviceAnimals', 'AskMen', 'AskReddit', 'BlackPeopleTwitter', 'CFB', 'DestinyTheGame', 'Games', 'IAmA', 'JUSTNOMIL', 'MMA', 'Overwatch', 'Showerthoughts', 'SquaredCircle', 'StarWars', 'The_Donald', 'WTF', 'WritingPrompts', 'aww', 'funny', 'gaming', 'gifs', 'hearthstone', 'hiphopheads', 'leagueoflegends', 'legaladvice', 'mildlyinteresting', 'movies', 'nba', 'news', 'nfl', 'nottheonion', 'pics', 'politics', 'relationships', 'soccer', 'technology', 'television', 'todayilearned', 'videos', 'worldnews']
40


In [3]:
file = open('data.train', 'w', buffering=10000)
labels = open('labels.train', 'w', buffering=10000)

In [4]:
for parent_id, reply_id, parent_body, parent_subreddit, parent_utc_time, parent_score, parent_length in ps2:
    cursor.execute("""SELECT * FROM replies WHERE id='{}'""".format(reply_id))
    reply = cursor.fetchone()
    if reply:
        reply_id, reply_body, reply_subreddit, reply_utc_time, reply_score, reply_length = reply     
        if parent_subreddit in labels_list:                 
            file.write(parent_body + '\n')
            file.write(reply_body + '\n') 
            labels.write(parent_subreddit +'\n')
            labels.write(reply_subreddit + '\n')            

In [5]:
file.close()
labels.close()

Κρατήσαμε μόνο τα σχόλια από τα 40 συνηθέστερα subreddits. Ας κάνουμε και cleaning:

In [9]:
a = string.printable + '’‘“”'
eng_char = set(a)
print(eng_char)

file = open('data.train')
clean_file = open('clean_data.train', 'w')

labels = open('labels.train')
clean_labels = open('clean_labels.train', 'w')

count = 0
count_clear = 0

{'R', 'N', 'Z', '”', '%', 'g', 'S', ';', '“', '_', '4', '$', 'F', '0', 'I', 'w', '@', 'M', ':', '\x0c', 'e', '5', 'P', 'y', 'K', 'W', '8', 'E', ',', '>', '\t', 'r', '"', '2', 'j', '\r', 'l', 'L', '\x0b', '7', 'z', 'C', '&', '(', '{', 'G', '3', '*', 't', 'q', 'X', 'H', 'o', '\n', '`', '-', 'm', 'T', '!', 'n', '\\', 'x', 'U', '^', '+', '/', 'Y', '=', 'c', '<', 'D', 'V', ' ', 'B', 'a', 'd', '~', '}', 'k', 'f', '‘', "'", '’', '[', '1', 'u', 'J', ']', '?', 'Q', 'h', '#', 'b', 'i', 'p', 'O', '9', 'v', '|', '6', 'A', 's', '.', ')'}


In [10]:
for f, l in zip(file, labels):
    count += 1
    if ('http' not in f) and (f != '\n'):
        comment = set(f)
        if comment.issubset(eng_char):
            count_clear +=1            
            while '....' in (f):
                f=f.replace('....','...')            
            clean_file.write(f)
            clean_labels.write(l)

print("Comment pairs before cleaning{}".format(count)) 
print("Clean Comment pairs{}".format(count_clear)) 

file.close()
labels.close()
clean_file.close()
clean_labels.close()

Comment pairs before cleaning631120
Clean Comment pairs608979


In [11]:
file2 = open('data.to','w')
for i in range(count_clear):
    file2.write('.\n')
print(count_clear)

file2.close()

608979


In [ ]:
Μεταφέρω το clean_data.train και το data.to  στο /home/myrto/pink_dir/github/myrto-tsokanaridou-chatbot/new_data και το μετονομάζω σε train.from

In [12]:
cd /home/myrto/pink_dir/github/myrto-tsokanaridou-chatbot/setup

/home/myrto/pink_dir/github/myrto-tsokanaridou-chatbot/setup


In [13]:
!python prepare_data.py


Preparing training set from raw set
File: tst.from
100%|#########################################| 1/1 [00:00<00:00, 10.33 lines/s]
File: dev.from
100%|#########################################| 1/1 [00:00<00:00,  9.13 lines/s]
File: train.from
 44%|############4               | 270000/608979 [00:09<00:12, 28159.53 lines/s]/home/myrto/p3venv/lib/python3.5/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|############################| 608979/608979 [00:20<00:00, 40948.56 lines/s]
File: tst.to
100%|#########################################| 1/1 [00:00<00:00,  5.67 lines/s]
File: dev.to
100%|#########################################| 1/1 [00:00<00:00, 11.00 lines/s]
File: train.to
100%|###########################| 608979/608979 [00:01<00:00, 327033.51 lines/s]

Learning BPE
Building temporary vocab (from)
195086 tokens [00:01, 110355.45 tokens/s]
Learning BPE fo